After EDA, what we need to do next are listed below:
1. Transfrom all string to int or date for later processing
2. Group train data with group_1 and extract those groups who have single outcome as first prediction column
3. Make a cross validation set based on people_id(delete all repeted date)
4. Make a group-date table and make a prediction on both train and test set via interpolation.(Second prediciton column, still have some None value)
5. Fill those none value with average score in train set(third prediction column)
6. Group dataset with group_1. Transform those categoty variables in the way of frequency.(#type in the group/# this group). Then we get a group table containing all those feature.
7. Mutate group date to create a new column called outcome_type.(0 represents all observations are 0, 1 representd all observation are 1, 2 represends mixed value.)
8. Make ensemble-based model to predict both cv and test set. At end of this step, we should have gotten a well-performed 4th prediction on both train and test set.


### 1st step

In [ ]:
import pandas as pd
import numpy as np
from operator import itemgetter
from itertools import product
from sklearn.metrics import roc_auc_score

In [ ]:
#read all data
people = pd.read_csv('people.csv')
train = pd.read_csv('act_train.csv')
test = pd.read_csv('act_test.csv')

In [ ]:
#String in dataset are in the form of "str int"
#We only want to keep the int part

#Get start with people
names = []
for name in people.columns:
    if 'char' in name:
        if type(people[name][0]) == str:
            people[name] = people[name].str.replace('type ','')
        if type(people[name][0]) == np.bool_:
            people[name] = people[name]+0
    if 'group' in name:
        people[name] = people[name].str.replace('group ', '')
    if 'people_id' in name:
        people[name] = people[name].str.replace('ppl_', '')
        names.append(name)
    else:
        names.append('ppl_' + name)

#transform date to date variable
people['date'] = pd.to_datetime(people['date'])

#change column names
people.columns = names

In [ ]:
#Then transform act dataset
#fill NA
train = train.fillna('type 0')
test = test.fillna('type 0')
test['outcome'] = None

train = train.append(test)

#transfrom date
train['date'] = pd.to_datetime(train['date'])

names = []
for name in train.columns:
    if name == 'outcome':
        names.append(name)
        continue
    if 'char' in name or 'category' in name:
        train[name] = train[name].str.replace('type ','')
    if 'activity_id' in name:
        train[name] = train[name].str.replace('act2_','')
        train[name] = train[name].str.replace('act1_','')
    if 'people_id' in name:
        train[name] = train[name].str.replace('ppl_', '')
        names.append(name)
    else:
        names.append('act_' + name)
        
#change columns name
train.columns = names

#split
test = train[train['outcome'].isnull()]
train = train[~train['outcome'].isnull()]

In [ ]:
#merge act and people
train = pd.merge(train, people, on = 'people_id')
test = pd.merge(test, people, on = 'people_id')

### 2ed step

In [ ]:
#Then we want to apply what we find in EDA: some groups only hava same outcome.
#To avoid occurance, we will choose those group who occued over 100 times.
all_zero_group = []
all_one_group = []
other_group = []
count = 0
#get different group and store them in list
for g in train.groupby('ppl_group_1'):
    if len(g[1]) > 100:
        outcome_mean = g[1]['outcome'].mean()
        if outcome_mean == 1:
            all_one_group.append( g[0] )
        elif outcome_mean == 0:
            all_zero_group.append( g[0] )
        else:
            other_group.append( g[0] )
            count += len(g[1])
    else:
        other_group.append( g[0] )
        count += len(g[1])

In [ ]:
data = train.append(test)
#get all leak data position
#leak_one = data['ppl_group_1'].isin(all_zero_group)
#leak_zero = data['ppl_group_1'].isin(all_one_group)
#data['outcome_leak1'] = data['outcome'].copy()
data['outcome_leak1'] = None
data['outcome_leak1'][data['ppl_group_1'].isin(all_zero_group)] = 0.05
data['outcome_leak1'][data['ppl_group_1'].isin(all_one_group)] = 0.95

#Split data into train and test again
train = data[~data['outcome'].isnull()]
test = data[data['outcome'].isnull()]

In [ ]:
#We want to build cross validation dataset
#By EDA, there are duplicated data in our dataset
#We want there are same number of valid observation in each fold.
names = []
for name in train.columns:
    if name != 'act_activity_id':
        names.append(name)
#unique_train contrains all valid information
unique_train = train[~train.duplicated(subset = names)]

In [ ]:
#Based on discussion on the forum, it's a good choice to create cv set by people_id
#cv_people contains people_id and its observation number and its mean score (used for stratify)
cv_people = []
for g in unique_train.groupby('people_id'):
    cv_people.append([g[0], len(g[1]), g[1]['outcome'].mean()])
#sort outcome mean since we want to get stratified set based on outcome
cv_people = sorted(cv_people, key = itemgetter(2), reverse = True)

In [ ]:
#Then create cv set based on id and corresponding number of non-repeated activity
#create 10-fold containing outcome stratified people_id
cv_people_id = []
cv_group_count = np.zeros(10)
for i in range(10):
    cv_people_id.append( [])
for g in cv_people:
    need_add_index = np.argsort(cv_group_count)[0]
    cv_people_id[need_add_index].append(g[0])
    cv_group_count[need_add_index] += g[1]

In [ ]:
#create train and cv stratified dataset
#cv_train contains unqiue information
#cv_eval contains all information
cv_train = []
cv_eval = []
cv_eval_tgt = []

#cv_eval_dup = []
#??
#cv_eval_tgt_dup = []

for g in cv_people_id:
    cv_train.append(unique_train[~unique_train.people_id.isin(g)])
    
    #cv_eval.append(unique_train[unique_train.people_id.isin(g)])
    #cv_eval_tgt.append(unique_train[unique_train.people_id.isin(g)][['act_activity_id', 'outcome']].copy())
    
    cv_eval.append(train[train.people_id.isin(g)])
    cv_eval_tgt.append(train[train.people_id.isin(g)][['act_activity_id', 'outcome'] ].copy())


In [ ]:
#According to EDA, combination of group and date map to unique outcome
#Thus, we want to create a group,date table
train['act_date'] = pd.to_datetime(train['act_date'])

alldays = pd.date_range(min(train['act_date']),  max(train['act_date']), freq='D')

In [ ]:
"""
Function from Kaggle Forum, Thanks to the provider
"""

def interpolateFun0(x):
    """Original script author's function rewritten in Python.
    The author interpolates between two known values by averaging them. We
    can think of this as 0th order interpolation. """

    ## TODO: This function could use some optimization. The R version is much faster...
    x = x.reset_index(drop=True)
    g = x['outcome'].copy() ## g should be a list or a pandas Series.
    
    global fv

    if (g.shape[0] < 3): ## If we have at most two rows.
        x['outcome_filled'] = g ## Will be replaced by a mean.
#        x['outcome'] = x['filled']
        return x
    
    if np.sum(g.isnull()) == 0:
        x['outcome_filled'] = g
        return x
    
    out = g.values.copy()
    value_locs = np.where(~g.isnull())[0]
    
    if len(value_locs) == 0:
        x['outcome_filled'] = np.full_like(out, np.nan)
#        x['outcome'] = x['filled']
        return x
    
    if len(value_locs) == 1:
        fillval = .89 if (g[value_locs[0]] == 1) else .13
        fv.append((g[value_locs[0]], fillval))
        g[g.isnull()] = fillval

        x['outcome_filled'] = g
#        x['outcome'] = x['filled']

        return x        
    
    # Fill in beginning (if needed)
    if value_locs[0]:
        
        fillval = .89 if (g[value_locs[0]] == 1) else .13
        fv.append((g[value_locs[0]], fillval))
        
        out[0:value_locs[0]] = fillval

    # Interpolate holes in the middle
    for i in range(0, len(value_locs) - 1):
        beg = value_locs[i]
        end = value_locs[i + 1]
        
        if g[beg] != g[end]:
            out[beg+1:end] = np.interp(range(beg+1, end), [beg, end], [g[beg], g[end]])
        else:
            out[beg+1:end] = g[beg]

    # Fill in end (if needed)
    if end < (len(g) - 1):
        beg = value_locs[-1]
        fillval = .89 if (g[beg] == 1) else .13
        fv.append((g[beg], fillval))

        out[beg+1:] = fillval

    x['outcome_filled'] = out
#    x['outcome'] = x['filled']
    
    return x

In [ ]:
#Apply this rule via interpolation on this each cross-validation set and check its power
fv = []
for i in range(10):
    #create group-date table for cv set used to be interpolation
    #get all group in a cv set
    group_name = cv_eval[i]['ppl_group_1'].unique()
    cv_gd_table = pd.DataFrame.from_records(product(group_name, alldays))
    cv_gd_table.columns = ['ppl_group_1','act_date']

    #get known gt-table in the train set
    train_gd_table = cv_train[i].groupby(['ppl_group_1', 'act_date'])['outcome'].agg('mean').to_frame().reset_index()
    train_gd_table['act_date'] = pd.to_datetime(train_gd_table['act_date'])
    #fill cv table by train table
    cv_gd_table = pd.merge(cv_gd_table, train_gd_table,on =['ppl_group_1','act_date'],  how = 'left')
    
    #insert value to the None date in the same group
    cv_gd_table = cv_gd_table.groupby('ppl_group_1').apply(interpolateFun0)
    cv_gd_table.columns = ['ppl_group_1', 'act_date', 'outcome_leak2', 'outcome_ip']
    
    cv_eval[i]['act_date'] = pd.to_datetime(cv_eval[i]['act_date'] )
    cv_eval[i] = pd.merge(cv_eval[i], cv_gd_table, on = ['ppl_group_1', 'act_date'], how = 'left')
    
    #Since sometimes, groups in cv set are not in train set. In this case, we can't fill th value. Instead, we can fill all Na with the
    #mean value in training set
    cv_eval[i]['outcome_filled'] = cv_eval[i]['outcome_ip'].fillna(cv_train[i]['outcome'].mean())

del  cv_gd_table
del train_gd_table

In [ ]:
#We should append all we get to trainset as new feature
#concat eval set together and merge with original train set
temp = pd.concat(cv_eval)[['act_activity_id','outcome_leak2', 'outcome_ip', 'outcome_filled']]
train = pd.merge(train, temp, on='act_activity_id', how = 'left')
del temp

In [ ]:
group_name = test['ppl_group_1'].unique()
cv_gd_table = pd.DataFrame.from_records(product(group_name, alldays))
cv_gd_table.columns = ['ppl_group_1','act_date']

train_gd_table = train.groupby(['ppl_group_1', 'act_date'])['outcome'].agg('mean').to_frame().reset_index()
train_gd_table['act_date'] = pd.to_datetime(train_gd_table['act_date'])

cv_gd_table = pd.merge(cv_gd_table, train_gd_table,on =['ppl_group_1','act_date'],  how = 'left')


cv_gd_table = cv_gd_table.groupby('ppl_group_1').apply(interpolateFun0)
cv_gd_table.columns = ['ppl_group_1', 'act_date', 'outcome_leak2', 'outcome_ip']

test['act_date'] = pd.to_datetime(test['act_date'] )
test = pd.merge(test, cv_gd_table, on = ['ppl_group_1', 'act_date'], how = 'left')

test['outcome_filled'] = test['outcome_ip'].fillna(train['outcome'].mean())

del  cv_gd_table
del train_gd_table

In [ ]:
#To fit our data into classifier, we need to transform our category variables via one-hot or tf-idf.
data = train.append(test)

#According to EDA, category variables include 
#act: "act_activity_category", 'act_char_1' --'act_char_10'
#people: 'ppl_group_1', 'ppl_group_1' -- 'ppl_group_9'
#'ppl_group_1' -'ppl_group_9' are the detailed activity, 'ppl_group_10' is more vague. Thus it containes more 
#much more tyep than any one of others. We need to filter some its value.
char_10_count = train.groupby('act_char_10')['ppl_char_10'].agg('count').order(ascending = False)

#transform all low-frequency type as -1
a = data.act_char_10.copy()
a[a.isin(char_10_count[char_10_count <= 100].index)] = -1
data.act_char_10 = a
del a

In [ ]:
#backup
data.act_date = pd.to_datetime(data.act_date).values.astype('datetime64[D]')
data.ppl_date = pd.to_datetime(data.ppl_date).values.astype('datetime64[D]')

In [ ]:
#manage date-related variable, mutate them to create more variables
#extract the weekday information
data['act_dayofweek'] = data['act_date'].copy()
data['act_dayofweek'] = data['act_dayofweek'].dt.dayofweek

#get the day difference starting with the minimum day

#first for act
minimum_day = data['act_date'].min()
data['act_date_diff'] = data['act_date'].copy()
data['act_date_diff'] = (data['act_date_diff'] - minimum_day) / np.timedelta64(1, 'D') 

#Then for people
minimum_day = data['ppl_date'].min()
data['ppl_date_diff'] = data['ppl_date'].copy()
data['ppl_date_diff'] = (data['ppl_date_diff'] - minimum_day) / np.timedelta64(1, 'D')
#Besides, we assume there may be some connection between ppl_date and corresponding act_ppl
data['ap_date_diff'] = (data['act_date'] - data['ppl_date']) /  np.timedelta64(1, 'D')

train = data[~data['outcome'].isnull()]
test = data[data['outcome'].isnull()]

In [ ]:
#When apply frequency method, avoid being affected by repeated observation. 
names = []
for name in data.columns:
    if name != 'act_activity_id':
        names.append(name)
#unique_train contrains all valid information
unique_data = data[~data.duplicated(subset = names)]

In [ ]:
#We transform category variable via group_1 and some group only occured in test set. So we don't know  
#the actual return value these groups. We want to extract these group names first.
test_groups = test.ppl_group_1[~test.ppl_group_1.isin(train.ppl_group_1.unique())].unique()

In [ ]:
#On the forum, they mentioned a important point that we should mutate more features within same group
#After we add date-related feature, it's time to do it.
mutate_features = {}
mutate_features['ppl_group_1'] = []
mutate_features['min_group_pdate'] = []
mutate_features['max_group_pdate'] = []
mutate_features['min_group_adate'] = []
mutate_features['max_group_adate'] = []
mutate_features['group_adate_range'] = []
mutate_features['group_ppl_number'] = []
mutate_features['group_act_number'] = []
mutate_features['group_adate_number'] = []

for g in data.groupby('ppl_group_1'):
    mutate_features['ppl_group_1'] .append( g[0] )
    mutate_features['min_group_adate'].append( g[1].act_date_diff.min())
    mutate_features['max_group_adate'].append( g[1].act_date_diff.max())
    mutate_features['min_group_pdate'].append( g[1].ppl_date_diff.min())
    mutate_features['max_group_pdate'].append( g[1].ppl_date_diff.max())
    mutate_features['group_adate_range'].append( g[1].act_date_diff.max() - g[1].act_date_diff.min())
    mutate_features['group_ppl_number'].append(len(g[1]['people_id'].unique()))
    mutate_features['group_act_number'].append(len(g[1]['act_activity_id'].unique()))
    mutate_features['group_adate_number'].append(len(g[1]['act_date'].unique()))
    
group_feature = pd.DataFrame(mutate_features)
#not necessary to merge
#data = pd.merge(data, ppl_in_group, on='ppl_group_1', how = 'left')


In [ ]:
#Then we want to transform these category variables to the form of frequency(general onehot aren't suitable to this)
#frequency is very sensitive to repeated observation.



#get all category variables' name
names = []
for i in data.columns:
    if 'char' in i and i != 'ppl_char_38' :
        names.append(i)
    elif i == 'act_activity_category':
        names.append(i)

In [ ]:
#we calculate type frequency within each group for each category variables
#create a function to do that:
def cal_freq(data, key):
    freq = {}
    #Get allunique possible value in this feature
    #col_names =  ['ppl_group_1'] + ('freq_' + data['ppl_char_5'].unique()).tolist()
    col_names = ['ppl_group_1'] + data[key].unique().tolist()
    a = data[key].unique().tolist()
    for n in col_names:
        freq[n] = []
    for g in data.groupby('ppl_group_1'):
        freq['ppl_group_1'].append(g[0])
        #get count and corresponding feature name
        c = a[:]
        value_count = g[1][key].value_counts()
        for tp in value_count.index:
            freq[tp].append(value_count[tp] / float(len(g[1][key])))
            c.remove(tp)
        for n in c:
            freq[n].append(0)
    freq = pd.DataFrame(freq)
    a = []
    for i in freq.columns:
        if i != 'ppl_group_1':
            if freq[i].mean > 0.001:
                i = key + "_"+str(i)
                a.append(i)
            else:
                freq = freq.drop(i)
        else:
            a.append(i)
    freq.columns = a
    return freq

In [ ]:
for key in names:
    freq = cal_freq(unique_data, key)
    group_feature = pd.merge(group_feature, freq, on= 'ppl_group_1', how = 'left')
    del freq

In [ ]:
#Then deal with char_38 which is continues variable
group_char_38 = {}
group_char_38['char_38_mean'] = []
group_char_38['char_38_median'] = []
group_char_38['char_38_var'] = []
group_char_38['ppl_group_1'] = []

#assign value
for g in unique_data.groupby('ppl_group_1'):
    group_char_38['ppl_group_1'].append(g[0])
    group_char_38['char_38_mean'].append(g[1]['ppl_char_38'].mean())
    group_char_38['char_38_median'].append(g[1]['ppl_char_38'].median())
    if len(g[1]) == 1:
        group_char_38['char_38_var'].append(0)
    else:
        group_char_38['char_38_var'].append(g[1]['ppl_char_38'].var())
group_char_38 = pd.DataFrame(group_char_38)
group_feature = pd.merge(group_feature,group_char_38 , on ='ppl_group_1', how = 'left')
del group_char_38

In [ ]:
group_train =group_feature[~group_feature.ppl_group_1.isin(test_groups)]
group_test =group_feature[group_feature.ppl_group_1.isin(test_groups)]

In [ ]:
#We have mutated all features we need. Then we'll divide this group dataframe into train and test. 
#We can't assign outcome directly for train set, since some group may have different value
#But we can transform to all0, all1 and mixed value for all groups
group_outcome = {}
group_outcome['ppl_group_1'] = []
group_outcome['outcome_type'] = []

for g in unique_data.groupby('ppl_group_1'):
    group_outcome['ppl_group_1'].append(g[0])
    m = g[1].outcome.mean()
    if m == 1:
        group_outcome['outcome_type'].append(1)
    elif m == 0:
        group_outcome['outcome_type'].append(0)
    else:
        group_outcome['outcome_type'].append(2)
group_outcome = pd.DataFrame(group_outcome)
group_train = pd.merge(group_train, group_outcome, on = 'ppl_group_1', how = 'left')
del group_outcome

In [ ]:
#By now, we have constructed train and test set. 
#Then create cv set
train_y = group_train['outcome_type']
train_X = group_train.drop('outcome_type', axis = 1)

from sklearn.cross_validation import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(train_y, 5, test_size=0.2, random_state=42)

cv_train_X, cv_train_y = {},{}
cv_test_X, cv_test_y = {},{}
cv_test_index = {}

i = 0
for train_index,test_index in sss:
    cv_train_X[i], cv_train_y[i] = train_X.iloc[train_index], train_y[train_index]
    cv_test_X[i], cv_test_y[i] = train_X.iloc[test_index], train_y[test_index]
    cv_test_index[i] = test_index
    i += 1

In [ ]:
#implement classifier on dataset, 5 classifer for 5 split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss
rf = {}
train_pred1 = []
for i in range(len(cv_train_X)):
    rf[i] = RandomForestClassifier(warm_start = True, random_state=42) #train same classifier multiple times
    for number in range(10,200, 20):
        rf[i].set_params(n_estimators = number)
        rf[i].fit(cv_train_X[i], cv_train_y[i])
    pred = rf[i].predict_proba(cv_test_X[i])
    print log_loss(cv_test_y[i], pred)
    pred = pd.DataFrame(pred, columns=['all0_pro', 'all1_pro', 'mixed_pro'])
    pred['ppl_group_1'] = train_X.iloc[cv_test_index[i]]['ppl_group_1']
    train_pred1.append(pred)

train_pred1 = pd.concat(train_pred1)

In [ ]:
import xgboost as xgb
from sklearn.metrics import log_loss

xg_train = {}
xg_test = {}
for i in range(5):
    xg_train[i] = xgb.DMatrix( cv_train_X[i], label= cv_train_y[i])
    xg_test[i] = xgb.DMatrix(cv_test_X[i], label= cv_test_y[i])

In [ ]:
# setup parameters for xgboost

param = {}
# use softmax multi-class classification
param['objective'] = 'multi:softprob'
# scale weight of positive examples
param['eta'] = 0.01
param['max_depth'] = 6
param['silent'] = 1
param['nthread'] = 4
param['num_class'] = 3
param['subsample'] = 0.7
param['colsample_bytree'] = 0.7
param['eval_metric'] = 'mlogloss'

bst = {}
pred = {}
for f in range(5):
    watchlist = [ (xg_train[f],'train'), (xg_test[f], 'test') ]
    num_round = 2500
    bst[f] = xgb.train(param, xg_train[f], num_round, watchlist , verbose_eval = 10, early_stopping_rounds=30)

In [ ]:
xgb_cv_pred = {}


from sklearn.metrics import log_loss

xgb_cv_preds = []
df_preds = []
for f in range(5):
    #preds = et[f].predict_proba(X_test[f])
    xgb_cv_pred[f] = bst[f].predict( xg_test[f], ntree_limit=bst[f].best_ntree_limit)
    df_preds.append(pd.DataFrame(xgb_cv_pred[f]))
    df_preds[-1].columns = ['gp_all0', 'gp_all1', 'gp_mixed']
    df_preds[-1]['ppl_group_1'] = train_X.iloc[cv_test_index[f]].ppl_group_1.values

df_preds_all = pd.concat(df_preds)


In [ ]:

gtesta = group_test.copy()
xgb_output = xgb.DMatrix(gtesta)
test_preds = []
for f in range(5):
    test_preds.append(bst[f].predict(xgb_output, ntree_limit = bst[f].best_ntree_limit))
    
preds_comb = np.mean([test_preds[j] for j in range(1,5)], axis = 0)

df_test_preds = pd.DataFrame(preds_comb)
df_test_preds.columns = ['gp_all0', 'gp_all1', 'gp_mixed']
df_test_preds['ppl_group_1'] = gtesta.ppl_group_1.values

In [ ]:
preds_out = pd.concat([df_test_preds, df_preds_all])

In [ ]:
preds_out.to_csv('group_pred.csv', index = False)

In [ ]:
train_first =train 
test_first = test("leak_test.csv") 
group_outcome = preds_out 
del train, test, preds_out

In [ ]:
#get a whole dataset
data = train_first.append(test_first)
data = pd.merge(data, group_outcome, on = 'ppl_group_1', how = 'left')

In [ ]:
data['leak_fillmask'] = data['outcome_ip'].isnull()
train = data[~data['outcome'].isnull()]
test = data[data['outcome'].isnull()]

train_1 = train[train_index['first_filter']].copy()

cols = train.columns.copy()
cols = cols.drop('act_activity_id')

train_1_dedup = train_1[~train_1.duplicated(subset = cols)]
#train_1_dup = train_1[train_1.duplicated(subset = cols)]

train_2 = train_1_dedup[~train_1_dedup['outcome_ip'].isnull()]

In [ ]:
from operator import itemgetter

ppl_count = []

for g in train_1_dedup.groupby('people_id'):
    ppl_count.append([g[0], len(g[1]), g[1].outcome.mean()])
ppl_count = sorted(ppl_count, key = itemgetter(2), reverse = True)

In [ ]:
cv_people_id = []
cv_group_count = np.zeros(5)
for i in range(5):
    cv_people_id.append( [])
for g in ppl_count:
    need_add_index = np.argsort(cv_group_count)[0]
    cv_people_id[need_add_index].append(g[0])
    cv_group_count[need_add_index] += g[1]

In [ ]:
cv_train_1 = []
cv_eval_1 = []
cv_train_2 = []
cv_eval_2 = []

cv_eval = []

for p in cv_people_id:
    cv_train_1.append(train_1_dedup[~train_1_dedup['people_id'].isin(p)])
    cv_eval_1.append(train_1_dedup[train_1_dedup['people_id'].isin(p)])
    cv_train_2.append(cv_train_1[-1][cv_train_1[-1]['outcome_ip'].isnull()])
    cv_eval_2.append(cv_eval_1[-1][cv_eval_1[-1]['outcome_ip'].isnull()])
    
    cv_eval.append(train_1[train_1['people_id'].isin(p)])

In [ ]:
unique_ppl = []
for g in cv_eval_1:
    unique_ppl.append(g['people_id'].unique())

In [ ]:
from scipy.sparse import csr_matrix, hstack

In [ ]:
def build_matrix(df):
    rows = []
    matrices = []
    
    rows.append(df['ppl_char_38'].values)
    rows.append(df['ap_date_diff'].values)
    rows.append(df['act_date_diff'].values)
    rows.append(df['ppl_date_diff'].values)
    
    rows.append(df['ppl_group_1'].values)
    rows.append(df['act_char_10'].values)
    
    rows.append(df['gp_all0'].values)
    rows.append(df['gp_all1'].values)
    rows.append(df['gp_mixed'].values)
    
    #rows.append(df['outcome_leak1'].values)
    #rows.append(df['outcome_ip'].values)
    
    matrices.append(csr_matrix(np.array(rows).T))
    return hstack(matrices)

In [ ]:
import xgboost as xgb

train_matrix = {}
eval_matrix = {}
train_dmat = {}
eval_dmat = {}

eval_all_matrix = {}
eval_all_dmat = {}

for i in range(5):
    train_matrix[i] = build_matrix(cv_train_2[i])
    eval_matrix[i] = build_matrix(cv_eval_2[i])
    eval_all_matrix[i] = build_matrix(cv_eval[i])
    
    train_dmat[i] = xgb.DMatrix(train_matrix[i], label=cv_train_2[i].outcome.values, missing=-1)
    eval_dmat[i] = xgb.DMatrix(eval_matrix[i], label=cv_eval_2[i].outcome.values, missing=-1)
    eval_all_dmat[i] = xgb.DMatrix(eval_all_matrix[i], label=cv_eval[i].outcome.values, missing=-1)
    
test_matrix = build_matrix(test)
dtest = xgb.DMatrix(test_matrix, missing=np.nan)

In [ ]:
#function from Kaggle Forum, Thanks to provider
def feval_procleak(yhat, y):
    if (len(yhat) != len(cv_val_dups[curfold])):
        return "auc", sklearn.metrics.roc_auc_score(y.get_label(), yhat)
    
    yhat_f = yhat.copy()
    
    locs = np.where(~cv_val_dups[curfold].leak_fillmask)
    yhat_f[locs] = cv_val_dups[curfold].outcome_filled.values[locs]
    
    return "auc", sklearn.metrics.roc_auc_score(y.get_label(), yhat_f)
from sklearn.metrics import roc_auc_score

def merge_pred(preds, cv_eval):
    cv_eval_pred = []
    for i in range(5):
        cv_eval_pred.append(cv_eval[i][['outcome','outcome_leak1', 'outcome_leak2', 'outcome_ip', 'outcome_filled', 'leak_fillmask']].copy())
        
        cv_eval_pred[i]['outcome_pred'] = preds[i]
        cv_eval_pred[i]['outcome_merge'] = preds[i]
        
        cv_eval_pred[i]['outcome_merge'] = cv_eval[i]['outcome_ip'][~cv_eval[i]['leak_fillmask']]
        
        print(i,
              roc_auc_score(cv_eval_pred[i]['outcome'].values, cv_eval_pred[i]['outcome_pred'].values),
              roc_auc_score(cv_eval_pred[i]['outcome'].values, cv_eval_pred[i]['outcome_merge'].values))
        
        return pd.concat(cv_eval_pred)

def pred_xgb(bst):
    preds = []
    for i in range(5):
        preds.append(bst[i].predict(eval_all_dmat[i], ntree_limit=bst[i].best_ntree_limit))
    
    output = merge_pred(preds, cv_eval)
    
    return output

In [ ]:
param = {'max_depth':10, 'eta':0.01, 'silent':1, 'objective':'binary:logistic' }
param['eval_metric'] = 'auc'
param['subsample'] = 0.5
param['colsample_bytree']= 0.3
param['min_child_weight'] = 1
param['max_depth'] = 5
param['booster'] = "gbtree"
param['seed'] = 12345

bst_d5 = {}

for curfold in range(5):
    watchlist  = [(train_dmat[curfold],'train'), (eval_dmat[curfold], 'eval')]
    num_round = 2000
    early_stopping_rounds=500
    bst_d5[curfold] = xgb.train(param, train_dmat[curfold], num_round, watchlist,
                       #feval = feval_procleak,
                       early_stopping_rounds=early_stopping_rounds, 
                       verbose_eval=50)

In [ ]:
preds_d5 = pred_xgb(bst_d5)